In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
from scipy import stats
import random

In [ ]:
# Hypothesis 1: Crimes are normal distributed 

In [ ]:

df = pd.read_csv("NYPD original.csv",delimiter=',')

In [ ]:
df = pd.read_csv("NYPD__dates_time_place.csv",delimiter=',')
print(df.columns)
df_From_Time = df[['From_Time']]
df_From_Time.set_index(['From_Time'])
print(df_From_Time.shape)

In [ ]:
# Hypothesis 1 : Data is normal distributed in a day
time = pd.DatetimeIndex(df['From_Time'])
all_times = time.hour * 60 + time.minute
print(all_times.shape)
time_bins = np.zeros(24)
x_bins = []
for i in range(24):
    x_bins.append(i)
for i in range(len(all_times)):
    time_bins[int(all_times[i] / 60)] = time_bins[int(all_times[i] / 60)] + 1

In [ ]:
D = pd.Series(x_bins)
series = pd.Series(time_bins, index=D)
pd.Series.plot(series)
ts = pd.Series(series)
ts.plot(kind="bar")
plt.show()
    

In [ ]:
shiftBy = 18
bins2 = []
for i in range(24):
    bins2.append(i)
    
bins2_roll = np.roll(bins2, shiftBy)
time_bins_rolled = np.roll(time_bins, shiftBy)
time_bins_rolled = time_bins_rolled/np.sum(time_bins_rolled)
D = pd.Series(bins2_roll)
series =pd.Series(time_bins_rolled, index=D)
pd.Series.plot(series)
ts = pd.Series(series)
ts.plot(kind="bar")
plt.ylabel("Normalized number of crimes per hour")
plt.xlabel("Time for a day(in 24 hour format)")
plt.show()

In [ ]:
#Hypothesis 1  - Task 1: Find mean and sigma using MLE and use K-S Test
print(time_bins_rolled)
n = len(time_bins_rolled)

mean_MLE = float(np.sum(time_bins_rolled))/n

sigma_sum = 0
for i in range(n):
    sigma_sum = sigma_sum +  (time_bins_rolled[i] - mean_MLE)*(time_bins_rolled[i] - mean_MLE)

sigma_MLE = np.sqrt(float(sigma_sum)/n)

print(": MLE mean ans sigma")
print(mean_MLE)
print(sigma_MLE)

time_bins_rolled_normalized_MLE = (time_bins_rolled - mean_MLE)/sigma_MLE
D, p = stats.kstest(time_bins_rolled_normalized_MLE, 'norm')

print("Max Distance:", D, ",  p-value: ", p)

se_hat = sigma_MLE/np.sqrt(n)

z_score = 1.96  # Two tailed for alpha = 0.05 i.e. 95% CI https://people.richland.edu/james/lecture/m170/ch08-int.html

CI_lower_range = mean_MLE - (z_score*se_hat)
CI_upper_range = mean_MLE + (z_score*se_hat)

print("CI_lower_range: ")
print(CI_lower_range)
print("CI_upper_range")
print(CI_upper_range)

In [ ]:
#Hypothesis 1  - Task 12: Find mean and sigma using MME and use K-S Test
print(time_bins_rolled)
n = len(time_bins_rolled)

mean_MME = float(np.sum(time_bins_rolled))/n

sigma_sum = 0
for i in range(n):
    sigma_sum = sigma_sum +  time_bins_rolled[i]*time_bins_rolled[i]

sigma_MME = np.sqrt(float(sigma_sum)/n - mean_MME*mean_MME)

print(": MME mean and sigma")
print(mean_MME)
print(sigma_MME)

time_bins_rolled_normalized_MME = (time_bins_rolled - mean_MME)/sigma_MME
D, p = stats.kstest(time_bins_rolled_normalized_MME, 'norm')

print("Max Distance:", D, ",  p-value: ", p)

In [ ]:
#Hypothesis 1  - Task 12: Find mean as 0.06 (max value)and sigma as distribution standard deviation and use K-S Test
print(time_bins_rolled)
n = len(time_bins_rolled)
mean = np.max(time_bins_rolled)#float(np.sum(time_bins_rolled))/n

sigma = np.std(time_bins_rolled)

print(": new mean ans sigma")
print(mean)
print(np.mean(time_bins_rolled))
print(sigma)

time_bins_rolled_normalized = (time_bins_rolled - mean)/sigma
D, p = stats.kstest(time_bins_rolled_normalized, 'norm')

print("Max Distance:", D, ",  p-value: ", p)

In [ ]:
#####SNAP Part
#Hypothesis: Food stamp doesn't affect the crime rate

In [ ]:
#read 2016 data
df2016 = pd.read_csv("NYPD_csv_2016.csv",delimiter=',')
df2016['date'] = pd.to_datetime(df2016.date).dt.date
print(df2016.shape)
print("***********")
df2016 = df2016.groupby(['date'], as_index=False).agg(['count'])
df2016 = df2016.reset_index()
print(df2016.shape)
print("***********")

In [ ]:

# Permutation Test
def permutationTest(bin_2016_first_15, bin_2016_last_16):
    Mean_first_15 = np.mean(bin_2016_first_15)
    Mean_last_16 = np.mean(bin_2016_last_16)
    
    print("First half mean: ")
    print(Mean_first_15)
    print("Second half mean: ")
    print(Mean_last_16)

    t_obs = math.fabs(Mean_first_15 - Mean_last_16)

    union = bin_2016[1:]
    ITERATIONS = 10000
    t_idx = []
    for idx in range(0, ITERATIONS):
        random.shuffle(union) # Step 2 (Permute)
        shuffle_first_15 = union[:15]
        shuffle_last_16 = union[15:]
        t_idx.append(math.fabs(np.mean(shuffle_first_15) - np.mean(shuffle_last_16))) # Step 3

    indicator_var = 0
    for each_t in t_idx:
        if each_t > t_obs:
            indicator_var += 1

    p_val = indicator_var / float(ITERATIONS)
    print("p-value using permutation Test: ", str(p_val))
    if p_val > 0.05: #(for 95% CI, alpha = 0.05)
        print ("Accepted, for hypothesis of equality")
    else:
        print ("Rejected, for hypothesis of equality")

In [ ]:
bin_2016 = np.zeros(32)
x_axis = []
for i in range(31):
    x_axis.append(i+1)


for index, row in df2016.iterrows():
    if(row['date'].apply(lambda x: x.month).values[0]) <= 6:
        temp = row['date'].apply(lambda x: x.day).values[0]
        bin_2016[temp] = bin_2016[temp] + row['count'].apply(lambda x: x).values[0]

print(sum(bin_2016))
series = pd.Series(bin_2016)
ts = pd.Series(series)
ts.plot(kind="bar")
plt.show()


def removeInvaliData(data):
    q75, q25 = np.percentile(data, [75 ,25])
    iqr = q75 - q25
    limit = 1.5*iqr
    lowerRange = q25 - limit
    upperRange = q75 + limit
    validData = []
    validData.append(0)
    for i in data:
        if i <= upperRange and i >= lowerRange:
            validData.append(i)
    return validData


bin_2016_new = removeInvaliData(bin_2016)
series = pd.Series(bin_2016_new)
ts = pd.Series(series)
ts.plot(kind="bar")
plt.show()

In [ ]:
bin_2016_first_15 = bin_2016[1:16]  # take 15 because SNAP was distributed till 15
bin_2016_last_16 = bin_2016[16:]

    
permutationTest(bin_2016_first_15, bin_2016_last_16)

# For 10000 Iterations
# p-value using permutation Test:  0.8132
# Accepted, for hypothesis of equality

# For 100000 Iterations
# p-value using permutation Test:  0.8506
# Accepted, for hypothesis of equality


In [ ]:
# Apply Permutation Test for each month from Jan 2016 - June 2016

for month in range(6):
    bin_2016 = np.zeros(32)
    x_axis = []
    for i in range(31):
        x_axis.append(i+1)


    for index, row in df2016.iterrows():
        if(row['date'].apply(lambda x: x.month).values[0]) == month + 1:
            temp = row['date'].apply(lambda x: x.day).values[0]
            bin_2016[temp] = bin_2016[temp] + row['count'].apply(lambda x: x).values[0]
    print(bin_2016)
    series = pd.Series(bin_2016)
    ts = pd.Series(series)
    ts.plot(kind="bar")
    plt.show()
#     bin_2016_31 = bin_2016[1:]
    print("\nFor month ", month + 1)
    bin_2016_first_15 = bin_2016[1:16]  # take 15 because SNAP was distributed till 15
    bin_2016_last_16 = bin_2016[16:]
    permutationTest(bin_2016_first_15, bin_2016_last_16)
    
    
    
    
# For 10,000 Iterations

# For month 1
# p-value using permutation Test:  0.643
# Accepted, for hypothesis of equality

# For month 2
# p-value using permutation Test:  0.4725
# Accepted, for hypothesis of equality

# For month 3
# p-value using permutation Test:  0.9461
# Accepted, for hypothesis of equality

# For month 4
# p-value using permutation Test:  0.589
# Accepted, for hypothesis of equality

# For month 5
# p-value using permutation Test:  0.7792
# Accepted, for hypothesis of equality

# For month 6
# p-value using permutation Test:  1.0
# Accepted, for hypothesis of equality



# For 100,000 Iterations

# For month 1
# p-value using permutation Test:  0.64242
# Accepted, for hypothesis of equality

# For month 2
# p-value using permutation Test:  0.46247
# Accepted, for hypothesis of equality

# For month 3
# p-value using permutation Test:  0.94577
# Accepted, for hypothesis of equality

# For month 4
# p-value using permutation Test:  0.59083
# Accepted, for hypothesis of equality

# For month 5
# p-value using permutation Test:  0.77284
# Accepted, for hypothesis of equality

# For month 6
# p-value using permutation Test:  1.0
# Accepted, for hypothesis of equality

In [ ]:
# Repeat Above two SNAP related Permutation Tests with removing Invalid Data(outliers)

def removeInvaliData(data):
    q75, q25 = np.percentile(data, [75 ,25])
    iqr = q75 - q25
    limit = 1.5*iqr
    lowerRange = q25 - limit
    upperRange = q75 + limit
    validData = []
    for i in data:
        if i <= upperRange and i >= lowerRange:
            validData.append(i)
    return validData

In [ ]:
# For 6 months average
bin_2016 = np.zeros(32)
x_axis = []
for i in range(31):
    x_axis.append(i+1)


for index, row in df2016.iterrows():
    if(row['date'].apply(lambda x: x.month).values[0]) <= 6:
        temp = row['date'].apply(lambda x: x.day).values[0]
        bin_2016[temp] = bin_2016[temp] + row['count'].apply(lambda x: x).values[0]

print(sum(bin_2016))

bin_2016_first_15_valid_data = removeInvaliData(bin_2016[1:16])  # take 15 because SNAP was distributed till 15
bin_2016_last_16_valid_data = removeInvaliData(bin_2016[16:])

permutationTest(bin_2016_first_15_valid_data, bin_2016_last_16_valid_data)

# For 10,000 Iterations
# p-value using permutation Test:  0.97
# Accepted, for hypothesis of equality

# For 100,000 Iterations
# p-value using permutation Test:  0.99989
# Accepted, for hypothesis of equality


In [ ]:
# Apply Permutation Test for each month from Jan 2016 - June 2016
for month in range(6):
    bin_2016 = np.zeros(32)
    x_axis = []
    for i in range(31):
        x_axis.append(i+1)


    for index, row in df2016.iterrows():
        if(row['date'].apply(lambda x: x.month).values[0]) == month + 1:
            temp = row['date'].apply(lambda x: x.day).values[0]
            bin_2016[temp] = bin_2016[temp] + row['count'].apply(lambda x: x).values[0]
            
#     bin_2016_31 = bin_2016[1:]
    print("\nFor month ", month + 1)
    bin_2016_first_15_valid_data = removeInvaliData(bin_2016[1:16])  # take 15 because SNAP was distributed till 15
    bin_2016_last_16_valid_data = removeInvaliData(bin_2016[16:])
    permutationTest(bin_2016_first_15_valid_data, bin_2016_last_16_valid_data)
    
    print(bin_2016_first_15_valid_data)
    print(bin_2016_last_16_valid_data)
    series = pd.Series(bin_2016_first_15_valid_data)
    ts = pd.Series(series)
    ts.plot(kind="bar")
    plt.show()
    series = pd.Series(bin_2016_last_16_valid_data)
    ts = pd.Series(series)
    ts.plot(kind="bar")
    plt.show()
    
    
    
# For 10,000 Iterations

# For month 1
# p-value using permutation Test:  0.7738
# Accepted, for hypothesis of equality

# For month 2
# p-value using permutation Test:  0.4639
# Accepted, for hypothesis of equality

# For month 3
# p-value using permutation Test:  0.9461
# Accepted, for hypothesis of equality

# For month 4
# p-value using permutation Test:  0.2645
# Accepted, for hypothesis of equality

# For month 5
# p-value using permutation Test:  0.8899
# Accepted, for hypothesis of equality

# For month 6
# p-value using permutation Test:  1.0
# Accepted, for hypothesis of equality



# For 100,000 Iterations

# For month 1
# p-value using permutation Test:  0.77462
# Accepted, for hypothesis of equality

# For month 2
# p-value using permutation Test:  0.46015
# Accepted, for hypothesis of equality

# For month 3
# p-value using permutation Test:  0.98428
# Accepted, for hypothesis of equality

# For month 4
# p-value using permutation Test:  0.26348
# Accepted, for hypothesis of equality

# For month 5
# p-value using permutation Test:  0.8874
# Accepted, for hypothesis of equality

# For month 6
# p-value using permutation Test:  0.99995
# Accepted, for hypothesis of equality